In [19]:
# Import Twitter API TOKEN
# import logging
# import sys
# sys.path.append('../../')

# logging.basicConfig(level=logging.INFO)
# logger = logging.getLogger()
from config import ConfigAPI
newconf = ConfigAPI()
api = newconf.create_api("auth1")



In [22]:
import tweepy
import pandas as pd

class StreamListener(tweepy.Stream):
    def __init__(self,api_key, api_secret, access_token, access_secret,limit=None):
        super().__init__(api_key,api_secret,access_token,access_secret)
        self.tweets = []
        if limit:
            self.limit= limit
        else:
            self.limit = 10
    
    def on_status(self, status):
        self.tweets.append(status)
        
        if len(self.tweets) == self.limit:
            self.disconnect()
        
    def on_error(self,status):
        print(status)

class SearchKeywordsInStream():
    listener = StreamListener(newconf.getKeys()[0],newconf.getKeys()[1],newconf.getKeys()[2],newconf.getKeys()[3])
        
    def filter_by_keywords(self,keywords):
        """Filter Twitter Stream by Keywords

        Args:
            keywords (list): Filter by list of keywords

        Returns:
            data (list): List of Tweets
        """
        self.listener.filter(track= keywords,languages=["en"])
        data = []
        for tweet in self.listener.tweets:
            if not tweet.truncated:
                data.append([tweet.text, tweet.created_at, tweet.user.followers_count, tweet.user.verified, tweet.user.created_at])
            else:
                data.append([tweet.extended_tweet, tweet.created_at, tweet.user.followers_count, tweet.user.verified, tweet.user.created_at])
        return data
    
    
#TODO: Extract Twitter history, 30days or max. days? or other day input and add to list
class TwitterHistory():
    pass
        
        
#TODO: Export this to a new File called DataHandling.py 
class ListToDF():
    """Converts a list to pandas Dataframe
    
    Args: 
        list (list): data
        columns (list): Title of Columns 
        
    Methods:
        export(type="Json" or "Excel, file_name="NameOfFile"): exports the Dataframe
        
        cleanDates(date=date): makes the date timezone unware so it can be exported to excel
    """
    def __init__(self,list,columns):
        pd.set_option("display.max_columns",100)
        self.df = pd.DataFrame(list,columns=columns)
        
    def export(self,type="Excel",file_name="exported_DataFrame"):
        """exports the Dataframe to Json or Excel

        Args:
            type (str, "Excel" or "Json"): Export to Excel or Json. Defaults to Excel.
            file_name (str, optional): Filename as String. Extension, .json or .xlsx, will be added automatically. Defaults to "exported_DataFrame".
        """
        if type=="Json":
            self.file_name = file_name + ".json"
            self.df.to_json(self.file_name)
        elif type == "Excel":
            self.df = self.df.applymap(self.cleanDates)
            self.file_name = file_name +".xlsx"
            self.df.to_excel(self.file_name)
            
        
    def cleanDates(self,date):
        """Cleans the Datetime Format by making it timezone unaware so it can be exported to Excel

        Args:
            date (str or datatime): Input. Use in .apply or .applymap for pandas Dataframes

        Returns:
            date: Returns cleaned date
        """
        import datetime
        self.date=date
        if isinstance(self.date,datetime.datetime):
            self.date = self.date.tz_localize(None)
            self.date = datetime.datetime.strftime(date,"%d-%m-%Y %H:%M")
        return self.date


In [23]:
#if __name__ == "main":
columns=["Tweet","Timestamp of Tweet","Following","Verified","User created"]

# Initalise a new Object of SearchKeywordsInStream and call the function filter_by_keywords
kws = SearchKeywordsInStream().filter_by_keywords(["btc","#btc"])

# Creating a new Object from ListToDF and add the list kws and columns as Args
df = ListToDF(kws,columns=columns)

# Exporting this list to Excel with filename 'exported_DataFrame.xlsx'
df.export()

# Print out the DataFrame
df.df



Stream connection closed by Twitter


,Tweet,Timestamp of Tweet,Following,Verified,User created
0,Binance - BTC Market\n#SC - Unusual buying act...,01-04-2022 13:44,96562,False,29-04-2019 20:48
1,DYDX\n\n Falling trend break and retest + cup ...,01-04-2022 13:44,0,False,01-04-2022 10:54
2,"NFT membership, NFT membership, NFT membership...",01-04-2022 13:44,226,False,16-02-2022 21:42
3,RT @EHAA80: @WorkQuest_co is a global end-to-e...,01-04-2022 13:44,2,False,16-10-2021 13:43
4,"RT @PiratesWorldNFT: 🤑 1,500 $BUSD GIVEAWAY 🤑\...",01-04-2022 13:44,1905,False,24-08-2011 08:43
5,@SpaceX @elonmusk #btc /#mtn money makes u...,01-04-2022 13:44,48,False,03-10-2021 13:32
6,RT @papa_woody21: Just got this tattoo so that...,01-04-2022 13:44,42,False,08-05-2021 06:29
7,"RT @ThxuToken: #Airdrop Campaign\nWinners: 2,0...",01-04-2022 13:44,42,False,18-11-2021 14:59
8,Signal 1624177 01-Apr 13:44 UTC\n\nHEART/BTC a...,01-04-2022 13:44,4726,False,01-05-2020 15:36
9,@WatcherGuru By now I’d should became clear to...,01-04-2022 13:44,59,False,07-10-2011 06:32
